Suppose that you would like to implement a tool that supports message writing, suggesting the next word while you are typing. Moreover, suppose that you would like the tool to learn from you or from a specific set of documents. Such a tool could be useful not only for providing message-writing assistance, but also for supporting spell checking, extracting common phrases, summarizing, and so on.

# Load Data

In [1]:
import os

from py2neo import Graph

# Connect to database
url = "bolt://localhost:7687"
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")
database = "masc"

graph = Graph(url, auth=(username, password), name=database)

In [ ]:

add_constraint = "CREATE CONSTRAINT FOR (w:Word) REQUIRE w.value IS UNIQUE;"
graph.run(add_constraint)

In [3]:
# doesn't work here use desktop
load_data = """
    :auto LOAD CSV FROM "file:///masc_sentences.tsv" AS line FIELDTERMINATOR '\t'
    CALL {
        WITH line
        WITH line[6] as sentence
        WITH split(sentence, " ") as words
        FOREACH ( idx IN range(0,size(words)-2) |
        MERGE (w1:Word {value:words[idx]})
        MERGE (w2:Word {value:words[idx+1]})
        MERGE (w1)-[r:NEXT]->(w2)
          ON CREATE SET r.weight = 1
          ON MATCH SET r.weight = r.weight + 1)
    } IN TRANSACTIONS OF 500 ROWS
    """
# Had to add masc_sentences.tsv to the DB server
graph.run(load_data)

ClientError: [Statement.SyntaxError] Invalid input ':': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FOREACH"
  "GRANT"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
":auto LOAD CSV FROM "file:///masc_sentences.tsv" AS line FIELDTERMINATOR '	'"
 ^